In [3]:
import torch
import torch.nn as nn

from functools import partial
from dataclasses import dataclass
from collections import OrderedDict

# Implementing ResNet in PyTorch
Today we are going to implement the famous ResNet from Kaiming He et al. (Microsoft Research). It won the 1st place on the ILSVRC 2015 classification task.

The original paper can be read from [here ](https://arxiv.org/abs/1512.03385) and it is very easy to follow, additional material can be found in this [quora answer](https://www.quora.com/)
![alt](./images/custom/rotated-resnet34.png)

*Deeper neural networks are more difficult to train.* Why? One big problem of deeper network is the vanishing gradient. Basically, the model is not able to learn anymore.

To solve this problem, the Authors proposed to use a reference to the previous layer to compute the output at a given layer. In ResNet, the output form the previous layer, called **residual**, is added to the output of the current layer. The following picture visualizes this operation

![alt](./images/residual.png)

We are going to make our implementation **as scalable as possible** using one think think unknown to mostly of the data scientiest: **object orienting programming**

## Basic Block

Okay, the first thing is to think about what we need. Well, first of all we need a convolution layer and since PyTorch does not have the 'auto' padding in Conv2d, so we have to code ourself!

In [4]:
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size
        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)      
        

In [5]:
conv = conv3x3(in_channels=32, out_channels=64)
print(conv)
del conv

Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


## Residual Block
To make clean code is mandatory to think about the main building block of each application, or of the network in our case. The residual block takes an input with `in_channels`, applies some blocks of convolutional layers to reduce it to `out_channels` and sum it up to the original input. If their sizes mismatch, then the input goes into an `identity`. We can abstract this process and create a interface that can be extedend.

In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.in_channels, self.out_channels =  in_channels, out_channels
        self.blocks = nn.Identity()
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels

In [7]:
ResidualBlock(32, 64)

ResidualBlock(
  (blocks): Identity()
  (shortcut): Identity()
)

Let's test it with a dummy vector with one one, we should get a vector with two

In [8]:
dummy = torch.ones((1, 1, 1, 1))

block = ResidualBlock(1, 64)
block(dummy)

tensor([[[[2.]]]])

In ResNet each block has a expansion parameter in order to increase the `out_channels`. Also, the identity is defined as a Convolution followed by an Activation layer, this is referred as `shortcut`. Then, we can just extend `ResidualBlock` and defined the `shortcut` function.

In [9]:
from collections import OrderedDict

class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(OrderedDict(
        {
            'conv' : nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            'bn' : nn.BatchNorm2d(self.expanded_channels)
            
        })) if self.should_apply_shortcut else None
        
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels

In [10]:
ResNetResidualBlock(32, 64)

ResNetResidualBlock(
  (blocks): Identity()
  (shortcut): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

### Basic Block
A basic ResNet block is composed by two layers of `3x3` convs/batchnorm/relu. In the picture, the lines represnet the residual operation. The dotted line means that the shortcut was applied to match the input and the output dimension.
![alt](./images/custom/Block.png)

Let's first create an handy function to stack one conv and batchnorm layer. Using `OrderedDict` to properly name each sublayer.

In [11]:
from collections import OrderedDict
def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(OrderedDict({'conv': conv(in_channels, out_channels, *args, **kwargs), 
                          'bn': nn.BatchNorm2d(out_channels) }))

In [12]:
conv_bn(3, 3, nn.Conv2d, kernel_size=3)

Sequential(
  (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (bn): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [13]:
class ResNetBasicBlock(ResNetResidualBlock):
    expansion = 1
    def __init__(self, in_channels, out_channels, activation=nn.ReLU, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            activation(),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),
        )
    

In [14]:
dummy = torch.ones((1, 32, 224, 224))

block = ResNetBasicBlock(32, 64)
block(dummy).shape
print(block)

ResNetBasicBlock(
  (blocks): Sequential(
    (0): Sequential(
      (conv): Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU()
    (2): Sequential(
      (conv): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (shortcut): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


### BottleNeck
To increase the network deepths but to decrese the number of parameters, the Authors defined a BottleNeck block that 
"The three layers are 1x1, 3x3, and 1x1 convolutions, where the 1×1 layers are responsible for reducing and then increasing (restoring) dimensions, leaving the 3×3 layer a bottleneck with smaller input/output dimensions." We can extend the `ResNetResidualBlock` and create these blocks.

In [15]:
class ResNetBottleNeckBlock(ResNetResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, activation=nn.ReLU, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
           conv_bn(self.in_channels, self.out_channels, self.conv, kernel_size=1),
             activation(),
             conv_bn(self.out_channels, self.out_channels, self.conv, kernel_size=3, stride=self.downsampling),
             activation(),
             conv_bn(self.out_channels, self.expanded_channels, self.conv, kernel_size=1),
        )
    

In [16]:
dummy = torch.ones((1, 32, 10, 10))

block = ResNetBottleNeckBlock(32, 64)
block(dummy).shape
print(block)

ResNetBottleNeckBlock(
  (blocks): Sequential(
    (0): Sequential(
      (conv): Conv2dAuto(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU()
    (2): Sequential(
      (conv): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): ReLU()
    (4): Sequential(
      (conv): Conv2dAuto(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (shortcut): Sequential(
    (conv): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


### Layer
A ResNet's layer is composed by blocks stacked one after the other. 

![alt](./images/custom/Layer.png)

We can easily defined it by just stuck `n` blocks one after the other, just remember that the first convolution block has a stide of two since "We perform downsampling directly by convolutional layers that have a stride of 2".

In [17]:
class ResNetLayer(nn.Module):
    def __init__(self, in_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        downsampling = 2 if in_channels != out_channels else 1
        
        self.blocks = nn.Sequential(
            block(in_channels , out_channels, *args, **kwargs, downsampling=downsampling),
            *[block(out_channels * block.expansion, 
                    out_channels, downsampling=1, *args, **kwargs) for _ in range(n - 1)]
        )

    def forward(self, x):
        x = self.blocks(x)
        return x

In [18]:
dummy = torch.ones((1, 32, 48, 48))

layer = ResNetLayer(64, 128, block=ResNetBasicBlock, n=3)
# layer(dummy).shape
layer

ResNetLayer(
  (blocks): Sequential(
    (0): ResNetBasicBlock(
      (blocks): Sequential(
        (0): Sequential(
          (conv): Conv2dAuto(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): ReLU()
        (2): Sequential(
          (conv): Conv2dAuto(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (shortcut): Sequential(
        (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ResNetBasicBlock(
      (blocks): Sequential(
        (0): Sequential(
          (conv): Conv2dAuto(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn

### Encoder
Similarly, the encoder is composed by multiple layer at increasing features size.

![alt](./images/custom/rotated-Encoder.png)



In [19]:
class ResNetEncoder(nn.Module):
    """
    ResNet encoder composed by increasing different layers with increasing features.
    """
    def __init__(self, in_channels=3, blocks_sizes=[64, 128, 256, 512], deepths=[2,2,2,2], 
                 activation=nn.ReLU, block=ResNetBasicBlock, *args,**kwargs):
        super().__init__()
        
        self.blocks_sizes = blocks_sizes
        
        self.gate = nn.Sequential(
            nn.Conv2d(in_channels, self.blocks_sizes[0], kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.blocks_sizes[0]),
            activation(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))
        self.blocks = nn.ModuleList([ 
            ResNetLayer(blocks_sizes[0], blocks_sizes[0], n=deepths[0], activation=activation, 
                        block=block,  *args, **kwargs),
            *[ResNetLayer(in_channels * block.expansion, 
                          out_channels, n=n, activation=activation, 
                          block=block, *args, **kwargs) 
              for (in_channels, out_channels), n in zip(self.in_out_block_sizes, deepths[1:])]       
        ])
        
        
    def forward(self, x):
        x = self.gate(x)
        for block in self.blocks:
            x = block(x)
        return x

## Decoder
The decoder is the last piece we need to create the full network. It is a fully connected layer that maps the features learned by the network to their respective classes. Easily, we can defined it as:

In [20]:
class ResnetDecoder(nn.Module):
    """
    This class represents the tail of ResNet. It performs a global pooling and maps the output to the
    correct class by using a fully connected layer.
    """
    def __init__(self, in_features, n_classes):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.decoder = nn.Linear(in_features, n_classes)

    def forward(self, x):
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        return x


## ResNet

Final, we can put all the pieces together and create the final model.

![alt](./images/custom/rotated-resnet34.png)

In [21]:
class ResNet(nn.Module):
    
    def __init__(self, in_channels, n_classes, *args, **kwargs):
        super().__init__()
        self.encoder = ResNetEncoder(in_channels, *args, **kwargs).to(device)
        self.decoder = ResnetDecoder(self.encoder.blocks[-1].blocks[-1].expanded_channels, n_classes).to(device)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

We can now defined the five models proposed by the Authors, `resnet18,34,50,101,152`

In [24]:
def resnet18(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBasicBlock, deepths=[2, 2, 2, 2])

def resnet34(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBasicBlock, deepths=[3, 4, 6, 3])

def resnet50(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 4, 6, 3])

def resnet101(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 4, 23, 3])

def resnet152(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 8, 36, 3])

In [23]:
from torchsummary import summary

model = resnet18(70, 2)
summary(model.cuda(), (3, 224, 224))

NameError: name 'device' is not defined

In [ ]:
import torchvision.models as models

# resnet101(False)

summary(models.resnet18(False).cuda(), (3, 224, 224))

In [25]:
import os
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import tqdm

In [26]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [27]:
db = "16.2"
features = "polar"
lag = "0"
model_save_dir = os.getcwd() + f"/ml-models/db{db}/{features}-lag{lag}/"

In [28]:
def loadNpy(filename):
    with open(os.getcwd() + "/train-val-test/"+ filename, "rb") as f: return np.load(f)

db = f"db{db}/window-size-10/lag{lag}/{features}-features/"
data_type = ""

In [29]:
DIR = os.getcwd() + "/train-val-test/" + db

In [78]:
X_train, X_val, X_test = loadNpy(db + f"{data_type}X_train.npy"), loadNpy(db + f"{data_type}X_val.npy"), loadNpy(db + f"{data_type}X_test.npy")
y_train, y_val, y_test = loadNpy(db + f"{data_type}y_train.npy"), loadNpy(db + f"{data_type}y_val.npy"), loadNpy(db + f"{data_type}y_test.npy")

In [79]:
minAcc = np.mean(y_train)
print(minAcc)

0.16762223283962413


In [80]:
y_train

array([0., 0., 0., ..., 0., 0., 1.])

In [81]:
def npy_to_tensor(data, l=False):
    if not l:
        data = torch.from_numpy(data).float()
    else:
        data = torch.from_numpy(data).type(torch.LongTensor)
    return data

In [82]:
X_train, X_val, X_test = npy_to_tensor(X_train), npy_to_tensor(X_val), npy_to_tensor(X_test)
y_train, y_val, y_test = npy_to_tensor(y_train, 1), npy_to_tensor(y_val, 1), npy_to_tensor(y_test, 1)

In [83]:
batch_size = 128

In [84]:
train_dataset = TensorDataset(X_train, y_train)

val_dataset = TensorDataset(X_val, y_val)

test_dataset = TensorDataset(X_test, y_test)

# Data loader
train_iterator = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)

val_iterator = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

test_iterator = DataLoader(dataset=test_dataset, batch_size=batch_size,shuffle=False)

In [85]:
X_train.shape

torch.Size([37674, 1, 28, 28])

In [86]:
loaders = {
    'train': train_iterator,
    'val': val_iterator,
    'test': test_iterator
}

In [92]:
from libauc.losses import AUCMLoss, SOAPLoss, APLoss_SH
from libauc.optimizers import PESG, SOAP_ADAM, SOAP_SGD
from sklearn.metrics import roc_auc_score

In [40]:
import torch.optim as optim

In [95]:
imratio = minAcc
SEED = 123
BATCH_SIZE = 64
lr =  1e-2
weight_decay = 2e-4
margin = 0.6
beta = 0.99 # this refers to gamma for moving average in the paper
posNum = 1

In [96]:
rn = resnet18(1, 2)#.to(device)
torch.manual_seed(SEED)

In [97]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(rn.parameters(), lr = 0.001, weight_decay=1e-3) 

In [48]:
loss_func = AUCMLoss(imratio=imratio)
loss_func

AUCMLoss()

In [36]:
optimizer = PESG(rn, 
                 a=loss_func.a, 
                 b=loss_func.b, 
                 alpha=loss_func.alpha, 
                 imratio=imratio, 
                 lr=lr, 
                 gamma=beta, 
                 margin=margin, 
                 weight_decay=weight_decay) #optim.Adam(rn.parameters(), lr = 0.0001, weight_decay=5e-2)   
optimizer

PESG (
Parameter Group 0
    a: tensor([0.], device='cuda:0', requires_grad=True)
    alpha: tensor([0.], device='cuda:0', requires_grad=True)
    b: tensor([0.], device='cuda:0', requires_grad=True)
    clip_value: 1.0
    gamma: 0.99
    lr: 0.01
    margin: 0.6
    model_acc: [tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0.,

In [98]:
swa_model = torch.optim.swa_utils.AveragedModel(rn)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)
swa_start = 160
swa_scheduler = optim.swa_utils.SWALR(optimizer, swa_lr=0.05)
sched_type = "swa"

In [ ]:
num_epochs = 2000
nn_epochs = {}

# Train the model
total_step = len(train_iterator)
for epoch in range(num_epochs):
    nn_epochs[epoch] = {}
    
    for phase in ["train", "val", "test"]:
        total_step = len(loaders[phase].dataset)
        correct = 0
        total = 0
        pred_ys = []
        labs = []
        
        if phase == "train":
            rn.train(True)
        else:
            rn.train(False)
    
        for i, (images, labels) in enumerate(loaders[phase]):  
            # Move tensors to the configured device
            #print(images.shape)
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            #print(images.shape)
            outputs = rn(images)
            #print(torch.max(outputs, 1)[1])
            pred_y = torch.max(outputs, 1)[1].data.squeeze()
            ##print(outputs.shape)
            #print(labels.shape)
            #print(pred_y.shape)
            #pred_y = torch.max(outputs, 1)[1].data.squeeze()
            loss = loss_func(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            
            if phase == "train":
                loss.backward()
                optimizer.step()
            #print(i)


            #print(pred_y)
            pred_ys = pred_ys + pred_y.tolist() #test_output.flatten().tolist()
            labs = labs + labels.tolist()
            total += len(labels)
            correct += (pred_y == labels).sum().item()

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

        print(f'Accuracy of the model on the {total} images: %.3f' % (correct/total))

        cm = np.zeros((2,2))
        for i, p in enumerate(pred_ys):
            label = labs[i]
            cm[label, p] += 1

        print(f"Sensitivity = {cm[1,1]/(cm[1,1] + cm[1,0])}")
        print(f"Specificity = {cm[0,0]/(cm[0,0] + cm[0,1])}")
        print(f"AUC = {roc_auc_score(labs, pred_ys)}")

        nn_epochs[epoch][phase] = {"loss": loss.item(), "acc": correct/total, 
                                 "sens": cm[1,1]/(cm[1,1] + cm[1,0]), "spec": cm[0,0]/(cm[0,0] + cm[0,1]),
                                "auc": roc_auc_score(labs, pred_ys)}

    print()

    if sched_type == "swa":
        if epoch > swa_start:
            swa_model.update_parameters(rn)
            swa_scheduler.step()
        else:
            scheduler.step()
    elif sched_type == "12":         
        scheduler1.step()
        scheduler2.step()

Epoch [1/2000], Step [295/37674], Loss: 0.5581
Accuracy of the model on the 37674 images: 0.842
Sensitivity = 0.34489311163895486
Specificity = 0.9426002104658949
AUC = 0.6437466610524248
Epoch [1/2000], Step [99/12558], Loss: 0.8541
Accuracy of the model on the 12558 images: 0.850
Sensitivity = 0.3922859830667921
Specificity = 0.9429639570552147
AUC = 0.6676249700610035
Epoch [1/2000], Step [99/12559], Loss: 0.5715
Accuracy of the model on the 12559 images: 0.843
Sensitivity = 0.37681818181818183
Specificity = 0.9415001448016218
AUC = 0.6591591633099019

Epoch [2/2000], Step [295/37674], Loss: 0.5348
Accuracy of the model on the 37674 images: 0.860
Sensitivity = 0.36832937450514647
Specificity = 0.9595331483784559
AUC = 0.6639312614418013
Epoch [2/2000], Step [99/12558], Loss: 0.8539
Accuracy of the model on the 12558 images: 0.847
Sensitivity = 0.21730950141110067
Specificity = 0.9748849693251533
AUC = 0.596097235368127
Epoch [2/2000], Step [99/12559], Loss: 0.5465
Accuracy of the mo

Epoch [15/2000], Step [99/12559], Loss: 0.5611
Accuracy of the model on the 12559 images: 0.846
Sensitivity = 0.15272727272727274
Specificity = 0.9934356598127232
AUC = 0.573081466269998

Epoch [16/2000], Step [295/37674], Loss: 0.3815
Accuracy of the model on the 37674 images: 0.876
Sensitivity = 0.4671417260490895
Specificity = 0.9581938199559935
AUC = 0.7126677730025415
Epoch [16/2000], Step [99/12558], Loss: 0.8100
Accuracy of the model on the 12558 images: 0.853
Sensitivity = 0.17262464722483536
Specificity = 0.9913726993865031
AUC = 0.5819986733056692
Epoch [16/2000], Step [99/12559], Loss: 0.4650
Accuracy of the model on the 12559 images: 0.848
Sensitivity = 0.17181818181818181
Specificity = 0.991794574765904
AUC = 0.581806378292043

Epoch [17/2000], Step [295/37674], Loss: 0.3707
Accuracy of the model on the 37674 images: 0.877
Sensitivity = 0.47030878859857483
Specificity = 0.9588315953952613
AUC = 0.7145701919969181
Epoch [17/2000], Step [99/12558], Loss: 1.2061
Accuracy of t

Epoch [30/2000], Step [99/12558], Loss: 0.7227
Accuracy of the model on the 12558 images: 0.864
Sensitivity = 0.34242709313264347
Specificity = 0.9704754601226994
AUC = 0.6564512766276713
Epoch [30/2000], Step [99/12559], Loss: 0.4162
Accuracy of the model on the 12559 images: 0.860
Sensitivity = 0.3277272727272727
Specificity = 0.9726807606911864
AUC = 0.6502040167092296

Epoch [31/2000], Step [295/37674], Loss: 0.3187
Accuracy of the model on the 37674 images: 0.887
Sensitivity = 0.5344418052256532
Specificity = 0.9580343760961766
AUC = 0.746238090660915
Epoch [31/2000], Step [99/12558], Loss: 0.4613
Accuracy of the model on the 12558 images: 0.868
Sensitivity = 0.431796801505174
Specificity = 0.9563842024539877
AUC = 0.6940905019795809
Epoch [31/2000], Step [99/12559], Loss: 0.3236
Accuracy of the model on the 12559 images: 0.861
Sensitivity = 0.41454545454545455
Specificity = 0.9563664446375133
AUC = 0.6854559495914839

Epoch [32/2000], Step [295/37674], Loss: 0.3252
Accuracy of th

Epoch [45/2000], Step [295/37674], Loss: 0.3397
Accuracy of the model on the 37674 images: 0.894
Sensitivity = 0.56215360253365
Specificity = 0.9606173666252112
AUC = 0.7613854845794307
Epoch [45/2000], Step [99/12558], Loss: 0.4453
Accuracy of the model on the 12558 images: 0.844
Sensitivity = 0.46378174976481656
Specificity = 0.921875
AUC = 0.6928283748824082
Epoch [45/2000], Step [99/12559], Loss: 0.3233
Accuracy of the model on the 12559 images: 0.838
Sensitivity = 0.46863636363636363
Specificity = 0.916883869099334
AUC = 0.6927601163678488

Epoch [46/2000], Step [295/37674], Loss: 0.2909
Accuracy of the model on the 37674 images: 0.895
Sensitivity = 0.5665874901029295
Specificity = 0.9612232532925157
AUC = 0.7639053716977227
Epoch [46/2000], Step [99/12558], Loss: 0.6206
Accuracy of the model on the 12558 images: 0.864
Sensitivity = 0.3019755409219191
Specificity = 0.9789110429447853
AUC = 0.6404432919333523
Epoch [46/2000], Step [99/12559], Loss: 0.4852
Accuracy of the model on t

Epoch [59/2000], Step [99/12559], Loss: 0.4481
Accuracy of the model on the 12559 images: 0.863
Sensitivity = 0.39636363636363636
Specificity = 0.9615793030215272
AUC = 0.6789714696925818

Epoch [60/2000], Step [295/37674], Loss: 0.3435
Accuracy of the model on the 37674 images: 0.898
Sensitivity = 0.5771971496437055
Specificity = 0.9625306929430147
AUC = 0.76986392129336
Epoch [60/2000], Step [99/12558], Loss: 0.5696
Accuracy of the model on the 12558 images: 0.872
Sensitivity = 0.5437441204139228
Specificity = 0.9386503067484663
AUC = 0.7411972135811946
Epoch [60/2000], Step [99/12559], Loss: 0.3192
Accuracy of the model on the 12559 images: 0.865
Sensitivity = 0.5259090909090909
Specificity = 0.937059561733758
AUC = 0.7314843263214245

Epoch [61/2000], Step [295/37674], Loss: 0.3205
Accuracy of the model on the 37674 images: 0.899
Sensitivity = 0.5814726840855107
Specificity = 0.963040913294429
AUC = 0.7722567986899698
Epoch [61/2000], Step [99/12558], Loss: 0.5782
Accuracy of the m

...

Epoch [463/2000], Step [99/12559], Loss: 1.6909
Accuracy of the model on the 12559 images: 0.825
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5

Epoch [464/2000], Step [295/37674], Loss: 0.5423
Accuracy of the model on the 37674 images: 0.837
Sensitivity = 0.18416468725257323
Specificity = 0.9683025606683887
AUC = 0.5762336239604811
Epoch [464/2000], Step [99/12558], Loss: 2.2445
Accuracy of the model on the 12558 images: 0.831
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5
Epoch [464/2000], Step [99/12559], Loss: 1.4641
Accuracy of the model on the 12559 images: 0.825
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5

Epoch [465/2000], Step [295/37674], Loss: 0.5191
Accuracy of the model on the 37674 images: 0.835
Sensitivity = 0.15407759303246238
Specificity = 0.9721929908479224
AUC = 0.5631352919401924
Epoch [465/2000], Step [99/12558], Loss: 0.6389
Accuracy of the model on the 12558 images: 0.831
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5
Epoch [465/2000], Step [99/12559], Loss: 0.497

Epoch [480/2000], Step [99/12558], Loss: 0.6230
Accuracy of the model on the 12558 images: 0.831
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5
Epoch [480/2000], Step [99/12559], Loss: 0.5020
Accuracy of the model on the 12559 images: 0.825
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5

Epoch [481/2000], Step [295/37674], Loss: 0.5044
Accuracy of the model on the 37674 images: 0.828
Sensitivity = 0.14505146476642913
Specificity = 0.9659427915430977
AUC = 0.5554971281547634
Epoch [481/2000], Step [99/12558], Loss: 1.1528
Accuracy of the model on the 12558 images: 0.831
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5
Epoch [481/2000], Step [99/12559], Loss: 0.8808
Accuracy of the model on the 12559 images: 0.825
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5

Epoch [482/2000], Step [295/37674], Loss: 0.5916
Accuracy of the model on the 37674 images: 0.826
Sensitivity = 0.16706254948535235
Specificity = 0.958480818903664
AUC = 0.5627716841945082
Epoch [482/2000], Step [99/12558], Loss: 9.7017

Epoch [497/2000], Step [99/12558], Loss: 4.9392
Accuracy of the model on the 12558 images: 0.831
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5
Epoch [497/2000], Step [99/12559], Loss: 3.6232
Accuracy of the model on the 12559 images: 0.825
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5

Epoch [498/2000], Step [295/37674], Loss: 0.5735
Accuracy of the model on the 37674 images: 0.832
Sensitivity = 0.15787806809184482
Specificity = 0.9679836729487548
AUC = 0.5629308705202998
Epoch [498/2000], Step [99/12558], Loss: 1.0016
Accuracy of the model on the 12558 images: 0.831
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5
Epoch [498/2000], Step [99/12559], Loss: 0.7938
Accuracy of the model on the 12559 images: 0.825
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5

Epoch [499/2000], Step [295/37674], Loss: 0.5882
Accuracy of the model on the 37674 images: 0.826
Sensitivity = 0.123673792557403
Specificity = 0.9679198954048279
AUC = 0.5457968439811155
Epoch [499/2000], Step [99/12558], Loss: 2.8675


Epoch [514/2000], Step [99/12558], Loss: 0.6317
Accuracy of the model on the 12558 images: 0.831
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5
Epoch [514/2000], Step [99/12559], Loss: 0.4996
Accuracy of the model on the 12559 images: 0.825
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5

Epoch [515/2000], Step [295/37674], Loss: 0.7231
Accuracy of the model on the 37674 images: 0.827
Sensitivity = 0.12684085510688836
Specificity = 0.9685257820721324
AUC = 0.5476833185895105
Epoch [515/2000], Step [99/12558], Loss: 1.2918
Accuracy of the model on the 12558 images: 0.831
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5
Epoch [515/2000], Step [99/12559], Loss: 0.8918
Accuracy of the model on the 12559 images: 0.825
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5

Epoch [516/2000], Step [295/37674], Loss: 0.5913
Accuracy of the model on the 37674 images: 0.817
Sensitivity = 0.12446555819477435
Specificity = 0.957013935393348
AUC = 0.5407397467940612
Epoch [516/2000], Step [99/12558], Loss: 2.4184

Epoch [531/2000], Step [99/12558], Loss: 0.7190
Accuracy of the model on the 12558 images: 0.831
Sensitivity = 0.0042333019755409216
Specificity = 0.9996165644171779
AUC = 0.5019249331963594
Epoch [531/2000], Step [99/12559], Loss: 0.5116
Accuracy of the model on the 12559 images: 0.826
Sensitivity = 0.006363636363636364
Specificity = 0.9999034655854813
AUC = 0.5031335509745588

Epoch [532/2000], Step [295/37674], Loss: 0.6605
Accuracy of the model on the 37674 images: 0.846
Sensitivity = 0.26999208234362626
Specificity = 0.9619566950476737
AUC = 0.61597438869565
Epoch [532/2000], Step [99/12558], Loss: 1.6215
Accuracy of the model on the 12558 images: 0.831
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5
Epoch [532/2000], Step [99/12559], Loss: 1.4659
Accuracy of the model on the 12559 images: 0.825
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5

Epoch [533/2000], Step [295/37674], Loss: 0.6134
Accuracy of the model on the 37674 images: 0.823
Sensitivity = 0.13745051464766428
Specificity

In [145]:
eval_rn(rn, "val")
eval_rn(rn, "test")

val Accuracy of the model on the 393 val images: 0.858
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5
test Accuracy of the model on the 393 test images: 0.878
Sensitivity = 0.0
Specificity = 1.0
AUC = 0.5


0.8778625954198473

In [ ]:
2

In [ ]:
num_epochs = 10
nn_epochs = {}

# Train the model
total_step = len(val_iterator)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(val_iterator):  
        # Move tensors to the configured device
        #print(images.shape)
        images = images.reshape(-1, 70, 70, 70).to(device)
        labels = labels.to(device)
        
        # Forward pass
        #print(images.shape)
        outputs = rn(images)
        pred_y = torch.max(outputs, 1)[1].data.squeeze()
        loss = loss_func(pred_y, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print(i)

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    nn_epochs[epoch] = {"loss": loss.item(), #"train": eval_rn(rn, "train"), 
                 "val": eval_rn(rn, "val"), "test": eval_rn(rn, "test")}
    rn.train()